In [1]:
!echo $USER | sudo -S systemctl restart nvargus-daemon

[sudo] password for jetson: 

In [2]:
%cd csc-3002-efficient-autonomous-obstacles-avoidance-on-jetracer-1-2025/notebooks/train_collision_avoidance

/home/jetson/csc-3002-efficient-autonomous-obstacles-avoidance-on-jetracer-1-2025/notebooks/train_collision_avoidance


# Build TensorRT model
- The purpose of this notebook is to optimize the neural network model developed in the `1_train_model_res18_3class.ipynb` notebook. It is assumed that the unoptimized neural network has already been created and loaded into the Jupyter Lab file browser. An untrained ResNet18 network with three outputs is loaded.

In [3]:
import cv2
import torch
import torchvision

In [4]:
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 3)
model = model.cuda().eval().half()

Next, load the trained weights from the `resnet18_3class.pth` file that you uploaded

In [5]:
model.load_state_dict(torch.load('resnet18_3class.pth'))

<All keys matched successfully>

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [6]:
device = torch.device('cuda')

### TensorRT
Convert and optimize the model using `torch2trt` for faster inference with TensorRT.

In [7]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below.

In [8]:
torch.save(model_trt.state_dict(), 'best_model_resnet18_3class_trt.pth')